In [2]:
import numpy as np
from sklearn import svm
import random
import tensorflow as tf
from sklearn import preprocessing
import skimage.io as io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mping
import os
from PIL import Image
import tensorflow as tf
import keras
from keras import backend as k
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Conv2DTranspose,merge
from skimage import morphology

In [3]:
X_train = np.load("C:/Users/cassi/Desktop/CT_input_all.npy")

In [5]:
X_train = X_train[:,200:450,150:400]

In [6]:
X_train.shape

(2847, 250, 250)

In [7]:
mask_train = np.load("C:/Users/cassi/Desktop/mask_input_all.npy")

In [8]:
index = np.where(mask_train == 1)[0]

In [9]:
index_unique = np.unique(index)

In [10]:
y_train = np.zeros(X_train.shape[0])
y_train[index_unique] = 1

In [11]:
num_category = 2
y_train = keras.utils.to_categorical(y_train, num_category)
image_number = X_train.shape[0]
split = int(0.8*image_number)

y_test = y_train[split:image_number,:]
y_input = y_train[0:split,:]

X_test = X_train[split:image_number,:,:]
X_input = X_train[0:split,:,:]
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]
if k.image_data_format() == 'channels_first':
    X_input = X_input.reshape(X_input.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_input = X_input.reshape(X_input.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [16]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape, padding='same', activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
 
model.add(Conv2D(32, (3, 3), padding='same', activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
 
model.add(Dropout(0.25)) 
model.add(Flatten()) 
model.add(Dense(64, activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(2, activation='softmax')) 
model.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['acc'])

In [20]:
X_input.shape

(2277, 250, 250, 1)

In [17]:
batch_size = 64
num_epoch = 5
model_log = model.fit(X_input,y_input,
                      batch_size = batch_size,
                      epochs = num_epoch,
                      verbose = 1,
                      validation_data = (X_test,y_test))

Instructions for updating:
Use tf.cast instead.
Train on 2277 samples, validate on 570 samples
Epoch 1/5
2277/2277 [==============================] - 222s 98ms/step - loss: 0.7916 - acc: 0.6930 - val_loss: 0.5779 - val_acc: 0.7281
Epoch 2/5
2277/2277 [==============================] - 225s 99ms/step - loss: 0.5481 - acc: 0.7251 - val_loss: 0.5433 - val_acc: 0.7281
Epoch 3/5
2277/2277 [==============================] - 213s 94ms/step - loss: 0.4745 - acc: 0.7404 - val_loss: 0.4639 - val_acc: 0.7579
Epoch 4/5
2277/2277 [==============================] - 207s 91ms/step - loss: 0.4445 - acc: 0.7523 - val_loss: 0.4663 - val_acc: 0.7737
Epoch 5/5
2277/2277 [==============================] - 209s 92ms/step - loss: 0.4100 - acc: 0.7703 - val_loss: 0.4766 - val_acc: 0.7719


In [18]:
X_predict = np.load("C:/Users/cassi/Desktop/CT_test.npy")

In [21]:
X_predict = X_predict[:,200:450,150:400]

In [22]:
X_predict = X_predict.reshape(X_predict.shape[0], img_rows, img_cols, 1)

In [24]:
y_predict = model.predict(X_predict)

In [37]:
size = y_predict.shape[0]
predict = np.zeros(size)
for i in range(size):
    predict[i] = np.argmax(y_predict[i])

In [45]:
predict

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.])

In [39]:
y_truth = np.load("C:/Users/cassi/Desktop/mask_test.npy")

In [40]:
y_index = np.unique(np.where(y_truth == 1)[0])

In [46]:
y_t_index = np.zeros(size)
y_t_index[y_index] = 1

In [50]:
1-sum(abs(predict-y_t_index))/size

0.8241758241758241